In [1]:
import getpass
import os
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_cohere import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
os.environ["COHERE_API_KEY"] = getpass.getpass()

 ········


In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

 ········


In [13]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [14]:
llm = ChatGroq(model="llama3-8b-8192")

In [15]:
bs4_strainer = bs4.SoupStrainer(attrs={'data-identity': ['headline', 'paragraph-element', 'unordered-list']})
loader = WebBaseLoader(
    web_paths=("https://my.clevelandclinic.org/health/diseases/15096-chronic-kidney-disease",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

15827

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

20

In [17]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=CohereEmbeddings(model='embed-english-v3.0'))

In [18]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the the common causes of chronic kidney disease?")

len(retrieved_docs)

6

In [52]:
print(retrieved_docs[0].page_content)

or needing to pee more often. Since symptoms vary, it’s best to call your healthcare provider if you believe there’s something wrong.What are common causes of kidney disease?Kidney diseases happen when your kidneys are damaged and can’t filter your blood. With chronic kidney disease, the damage tends to happen over the course of several years.High blood pressure (hypertension) and diabetes are the two most common causes of chronic kidney disease. Other causes and conditions that affect kidney function and can cause chronic kidney disease include:Glomerulonephritis. This type of kidney disease involves damage to the glomeruli, which are the filtering units inside your kidneys.Polycystic kidney disease. This is a genetic disorder that causes many fluid-filled cysts to grow in your kidneys, reducing the ability of your kidneys to function.Membranous nephropathy. This is a disorder where your body’s immune system attacks the waste-filtering membranes in your kidney.Obstructions of the


In [19]:
from langchain import hub

# Pull the RAG prompt from the hub
prompt = hub.pull("rlm/rag-prompt")

# Define the input information (model results and user inputs)
model_results = {
    "classification": "Chronic Kidney Disease - Stage 3",
    "GFR": "45 ml/min/1.73m²",  # Example result
    "creatinine": "2.1 mg/dL",
    "blood_pressure": "140/90 mmHg",
    "age": 55,
    "gender": "Male",
    "symptoms": ["fatigue", "swelling in legs", "high blood pressure"]
}

# Define the prompt message as a doctor creating a report based on the model's prediction and user inputs
doctor_prompt_message = f"""
You are a nephrologist. Based on the following patient data and model predictions, write a concise medical report for the patient:

- Diagnosis: {model_results['classification']}
- Glomerular Filtration Rate (GFR): {model_results['GFR']}
- Creatinine Levels: {model_results['creatinine']}
- Blood Pressure: {model_results['blood_pressure']}
- Age: {model_results['age']}
- Gender: {model_results['gender']}
- Symptoms: {', '.join(model_results['symptoms'])}

The report should include an assessment of the patient's condition, potential treatment options, and recommended follow-up actions.
"""

# Invoke the prompt with the doctor-style message
example_messages = prompt.invoke(
    {"context": doctor_prompt_message, "question": ""}
).to_messages()

# Display the generated messages
example_messages

/home/thepresiding/Desktop/Projects/kdpredictor/docReport/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion:  \nContext: \nYou are a nephrologist. Based on the following patient data and model predictions, write a concise medical report for the patient:\n\n- Diagnosis: Chronic Kidney Disease - Stage 3\n- Glomerular Filtration Rate (GFR): 45 ml/min/1.73m²\n- Creatinine Levels: 2.1 mg/dL\n- Blood Pressure: 140/90 mmHg\n- Age: 55\n- Gender: Male\n- Symptoms: fatigue, swelling in legs, high blood pressure\n\nThe report should include an assessment of the patient's condition, potential treatment options, and recommended follow-up actions.\n \nAnswer:", additional_kwargs={}, response_metadata={})]

In [11]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What are some causes of chronic kidney disease?"):
    print(chunk, end="", flush=True)

According to the provided context, common causes of chronic kidney disease (CKD) include high blood pressure (hypertension) and diabetes, as well as other conditions such as glomerulonephritis, polycystic kidney disease, membranous nephropathy, and obstructions of the urinary tract.